In [ ]:
from data_d_dicts import links
definitions = pd.read_csv(links['google_definition_csv'])
definitions

In [129]:
pd.read_csv(

,Date,Action,Result,Temp_Score,Historical_Score,Word,Process,Categorization,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,2026-01-07,Test,Fail,0,-3,SGDClassifier,ML Model,Algorithm,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
1,2026-01-07,Update,Push,0,-3,VotingClassifier,ML Model,Algorithm,VotingClassifier is an ensemble method that co...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Classification,Meta-Model,1.0
2,2026-01-07,Update,Push,0,-3,Quantile Random Forest,ML Model,Algorithm,Quantile Random Forest is an extension of rand...,NaN,NaN,NaN,NaN,NaN,Supervised,Regression,Tree,NaN
3,2026-01-07,Update,Push,0,-3,RandomTreesEmbedding,ML Model,Algorithm,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
4,2026-01-07,Test,Fail,0,-3,Area Under the Curve,Model Evaluation,Feature Selection,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,Test,Push,0,-3,Perceptron,ML Model,Algorithm,The Perceptron is one of the earliest linear c...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
6,2026-01-08,Test,Push,0,-3,RidgeCV,ML Model,Algorithm,RidgeCV is a regression model that combines ri...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Regression,Linear,NaN
7,2026-01-08,Update,Fail,0,-3,Regularization,ML Model,Definition,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,Test,Push,0,-3,SelectFpr,ML Model,Algorithm,SelectFpr is a feature selection method that s...,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
9,2026-01-08,Test,Push,0,-3,AdaBoostClassifier,ML Model,Algorithm,AdaBoostClassifier is an ensemble boosting alg...,NaN,NaN,NaN,NaN,"Small, Medium",Supervised,Classification,Linear,1.0


In [128]:
temp.to_csv('/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv',index=False)
temp = review_test_results()  
temp

Process: ML Model
Classification: Algorithm
Word: VotingClassifier

#############################################################################################################################


KeyboardInterrupt: Interrupted by user

In [19]:
df = review_test_results()
df.rename(columns={'Score':'OG_Score','Category':'Process'},inplace=True)
df['Review_Score'] = 0

In [22]:
definitions[definitions['Word'].str.contains('SGD')]

,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
199,ML Model,Algorithm,SGDClassifier,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
200,ML Model,Algorithm,SGDOneClassSVM,NaN,NaN,NaN,NaN,NaN,"Medium, Large",NaN,NaN,NaN,NaN
201,ML Model,Algorithm,SGDRegressor,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN


In [149]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

#from daily_processes import review_test_results,daily_test
#review_test_results()
#daily_test()







final_df, today_test,result_dict = daily_test()

Word 1
Category: Training
Classification: Optimizer
Word: Adagrad



######################### ANSWER QUESTION ######################### f


Definition: Variation of Gradient Descent, which adapts the learning rate for each parameter by scaling it inversely proportional to the sum of past squared gradients. 

Notes: 
Link: 
Markdown: $$
\theta_{t+1} = \theta_t - \frac{\alpha}{\sqrt{G_t + \epsilon}} \nabla J(\theta_t)
$$

where:
- $( G_t = \sum_{i=1}^{t} \nabla J(\theta_i) \odot \nabla J(\theta_i) )$ (element-wise sum of squared gradients),
- $( \epsilon )$ is a small constant to prevent division by zero.
Dataset Size: 
Learning Type: 
Alogrithm Class: 



What was the result? (P/F)
 f


Word 2
Category: ML Model
Classification: Algorithm
Word: NMF



...

######################### ANSWER QUESTION ######################### f


Definition: Representing metrics of directionality, momentum or variability, including periodic changes, standard deviations, volatility, or growth percentages. 
Notes: 
Link: 
Markdown: 
Dataset Size: 
Learning Type: 
Alogrithm Class: 



What was the result? (P/F)
 f


Word 5
Category: Graph Theory
Classification: Definition
Word: Grapth Theory



Update Google Sheet with Definition
 


In [150]:
final_df

,Date,Temp_Score,Historical_Score,Word,Process,Categorization,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,2026-01-07,0,-3,SGDClassifier,ML Model,Algorithm,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
1,2026-01-07,0,-3,VotingClassifier,ML Model,Algorithm,VotingClassifier is an ensemble method that co...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Classification,Meta-Model,1.0
2,2026-01-07,0,-3,Quantile Random Forest,ML Model,Algorithm,Quantile Random Forest is an extension of rand...,NaN,NaN,NaN,NaN,NaN,Supervised,Regression,Tree,NaN
3,2026-01-07,0,-3,RandomTreesEmbedding,ML Model,Algorithm,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
4,2026-01-07,0,-3,Area Under the Curve,Model Evaluation,Feature Selection,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,0,-3,Perceptron,ML Model,Algorithm,The Perceptron is one of the earliest linear c...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
6,2026-01-08,0,-3,RidgeCV,ML Model,Algorithm,RidgeCV is a regression model that combines ri...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Regression,Linear,NaN
7,2026-01-08,0,-3,Regularization,ML Model,Definition,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,0,-3,SelectFpr,ML Model,Algorithm,SelectFpr is a feature selection method that s...,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
9,2026-01-08,0,-3,AdaBoostClassifier,ML Model,Algorithm,AdaBoostClassifier is an ensemble boosting alg...,NaN,NaN,NaN,NaN,"Small, Medium",Supervised,Classification,Linear,1.0


In [151]:
today_test

,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble,Date,Historical_Score,Temp_Score
0,Training,Optimizer,Adagrad,"Variation of Gradient Descent, which adapts th...",,,,$$\n\theta_{t+1} = \theta_t - \frac{\alpha}{\s...,,,,,,2026-01-13,-3,0
1,ML Model,Algorithm,NMF,,,,,,"Medium, Large",,,,,2026-01-13,-2,0
2,Data Engineering,Definition,Cache,Cache takes load away from critical systems. D...,,,,,,,,,,2026-01-13,-3,0
3,Data Preparation,Semantic Type,Diagnostic,"Representing metrics of directionality, moment...",,,,,,,,,,2026-01-13,-3,0
4,Graph Theory,Definition,Grapth Theory,,,,,,,,,,,2026-01-13,-2,0


,Date,Temp_Score,Historical_Score,Word,Process,Categorization,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,2026-01-07,0,-3,SGDClassifier,ML Model,Algorithm,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
1,2026-01-07,0,-3,VotingClassifier,ML Model,Algorithm,VotingClassifier is an ensemble method that co...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Classification,Meta-Model,1.0
2,2026-01-07,0,-3,Quantile Random Forest,ML Model,Algorithm,Quantile Random Forest is an extension of rand...,NaN,NaN,NaN,NaN,NaN,Supervised,Regression,Tree,NaN
3,2026-01-07,0,-3,RandomTreesEmbedding,ML Model,Algorithm,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
4,2026-01-07,0,-3,Area Under the Curve,Model Evaluation,Feature Selection,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,0,-3,Perceptron,ML Model,Algorithm,The Perceptron is one of the earliest linear c...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
6,2026-01-08,0,-3,RidgeCV,ML Model,Algorithm,RidgeCV is a regression model that combines ri...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Regression,Linear,NaN
7,2026-01-08,0,-3,Regularization,ML Model,Definition,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,0,-3,SelectFpr,ML Model,Algorithm,SelectFpr is a feature selection method that s...,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
9,2026-01-08,0,-3,AdaBoostClassifier,ML Model,Algorithm,AdaBoostClassifier is an ensemble boosting alg...,NaN,NaN,NaN,NaN,"Small, Medium",Supervised,Classification,Linear,1.0


In [140]:
today_test['Historical_Score'] = today_test['Word'].map(result_dict)
today_test['Temp_Score']=0 
today_test['Action'] = 'Fail'
today_test

,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble,Date,Action,Result,Historical_Score,Temp_Score
0,Behavioural Economics,Definition,Blind spot bias,Cognitive bias where people are unaware of the...,,,,,,,,,,2026-01-13,Fail,,-3,0
1,Data Preparation,Semantic Type,Descriptive,Representing aggregated summarizations over mo...,,,,,,,,,,2026-01-13,Fail,,-3,0
2,Mathematics,Concept,Independence,Observations should be independent of each oth...,Updated REquired as more to BLUE,,,,,,,,,2026-01-13,Fail,,-3,0
3,Mathematics,Concept,Linearity,Relationship between Independent and Dependent...,Updated REquired as more to BLUE,,,,,,,,,2026-01-13,Fail,,-3,0
4,ML Model,Algorithm,Decision Trees,An ensemble of decision trees using bagging to...,,,,,,,,,,2026-01-13,Fail,,-3,0


In [135]:
final_df

,Date,Action,Result,Temp_Score,Historical_Score,Word,Process,Categorization,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,2026-01-07,Test,Fail,0,-3,SGDClassifier,ML Model,Algorithm,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
1,2026-01-07,Update,Push,0,-3,VotingClassifier,ML Model,Algorithm,VotingClassifier is an ensemble method that co...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Classification,Meta-Model,1.0
2,2026-01-07,Update,Push,0,-3,Quantile Random Forest,ML Model,Algorithm,Quantile Random Forest is an extension of rand...,NaN,NaN,NaN,NaN,NaN,Supervised,Regression,Tree,NaN
3,2026-01-07,Update,Push,0,-3,RandomTreesEmbedding,ML Model,Algorithm,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
4,2026-01-07,Test,Fail,0,-3,Area Under the Curve,Model Evaluation,Feature Selection,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,Test,Push,0,-3,Perceptron,ML Model,Algorithm,The Perceptron is one of the earliest linear c...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
6,2026-01-08,Test,Push,0,-3,RidgeCV,ML Model,Algorithm,RidgeCV is a regression model that combines ri...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Regression,Linear,NaN
7,2026-01-08,Update,Fail,0,-3,Regularization,ML Model,Definition,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,Test,Push,0,-3,SelectFpr,ML Model,Algorithm,SelectFpr is a feature selection method that s...,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
9,2026-01-08,Test,Push,0,-3,AdaBoostClassifier,ML Model,Algorithm,AdaBoostClassifier is an ensemble boosting alg...,NaN,NaN,NaN,NaN,"Small, Medium",Supervised,Classification,Linear,1.0


In [ ]:
from daily_processes import generate_dictionary
d_outline,examine_further  = generate_dictionary()

In [ ]:
######################################

# Action Add DQ Item.
# Add Read Me, which Explanation of process and Definitions of all Categorizations.

# DQ Validation Component

# Question I need to ask Myself.
# Is it a Definition.
# Is it a Step in a Process
# Is it a Process
# Is it information Related to a Process.

# DQ Process 1: Need to Avoid Duplication of Word in ALL SHEETS.
# Definitions: Are Technical Definitions, descriptions and eloborate Notes describing a Term. 
# Notes: Merge in Definitions.
# Take Longer Listings and organize them, Models, BLUE, 

######################################

In [ ]:
question_to_gpt = 'Can you provide me a 3-4 Sentence definition highlighting the definition, its origin, its importance and its application. If it is a model can you define >>>>>>'

In [ ]:
def generate_read_me():

    from data_d_dicts import links
    df = pd.read_csv(links['google_definition_csv'])

    read_me_df = pd.DataFrame(df.columns,columns=['W'])

    return read_me_df
    

In [ ]:
read_me_df = pd.DataFrame(df.columns,columns=['Word'])
read_me_df.merge(df[['Category','Categorization','Definition','Word']],on='Word',how='left')

read_me_df = pd.DataFrame(df.columns,columns=['Word'])
read_me_df


In [142]:
df.head(1)

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
file_location='/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv'
df = pd.read_csv(file_location)
df

,Date,Action,Result,Score,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
0,2026-01-07,Test,Fail,-1,Model,Algorithm,SGDClassifier,Handle Large Dataset well because it treats ea...,NaN,NaN,NaN,NaN,Large,NaN,NaN
1,2026-01-07,Update,Push,0,Model,Algorithm,VotingClassifier,NaN,NaN,NaN,NaN,NaN,"Medium, Large",NaN,NaN
2,2026-01-07,Update,Push,0,Model,Algorithm,Quantile Random Forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-07,Update,Push,0,Model,Algorithm,RandomTreesEmbedding,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN
4,2026-01-07,Test,Fail,-1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,Test,Push,0,Model,Algorithm,Perceptron,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN
6,2026-01-08,Test,Push,0,Model,Algorithm,RidgeCV,NaN,NaN,NaN,NaN,NaN,"Medium, Large",NaN,NaN
7,2026-01-08,Update,Fail,-1,Machine Learning,Definition,Regularization,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,Test,Push,0,Model,Algorithm,SelectFpr,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN
9,2026-01-08,Test,Push,0,Model,Algorithm,AdaBoostClassifier,NaN,NaN,NaN,NaN,NaN,"Small, Medium",NaN,NaN


NameError: name 'review_test_results' is not defined